In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
import librosa
from google.colab import drive
import os

# Mount Google Drive for access
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load data
with open('/content/drive/MyDrive/My-207/X_train_centroid.json', 'r') as file:
    X_train_list = json.load(file)
    X_train_centroid = np.array(X_train_list)

with open('/content/drive/MyDrive/My-207/y_train_centroid.json', 'r') as file:
    y_train_list = json.load(file)
    y_train = np.array(y_train_list)

# Load data from JSON files validation with centroid features
with open('/content/drive/MyDrive/My-207/X_val_centroid.json', 'r') as file:
    X_val_list = json.load(file)
    X_val_centroid = np.array(X_val_list)

with open('/content/drive/MyDrive/My-207/y_val_centroid.json', 'r') as file:
    y_val_list = json.load(file)
    y_val = np.array(y_val_list)


In [ ]:
# Data normalization
scaler = StandardScaler()

# Fit and transform the training data
X_train_centroid_normalized = scaler.fit_transform(X_train_centroid)

# Transform the validation data using the scaler parameters from the training data
X_val_centroid_normalized = scaler.transform(X_val_centroid)

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# One-hot encode the labels for LSTM
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)

# Save the label encoder for later use
label_encoder_path = "label_encoder.pkl"
import joblib
joblib.dump(label_encoder, label_encoder_path)

# Reshape the data for LSTM (add additional dimension for time steps)
X_train_lstm = X_train_centroid_normalized.reshape(X_train_centroid_normalized.shape[0], X_train_centroid_normalized.shape[1], 1)
X_val_lstm = X_val_centroid_normalized.reshape(X_val_centroid_normalized.shape[0], X_val_centroid_normalized.shape[1], 1)



In [ ]:
# Build LSTM model topology for centroid features
model = tf.keras.Sequential([
    LSTM(128, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=True),
    LSTM(64),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])






In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 251, 128)          66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 116,163
Trainable params: 116,163
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_lstm, y_train_onehot, validation_data=(X_val_lstm, y_val_onehot),
                    batch_size=32, epochs=100, callbacks=[early_stopping])


Epoch 1/100
143/143 [==============================] - 102s 642ms/step - loss: 1.0590 - accuracy: 0.4536 - val_loss: 1.1556 - val_accuracy: 0.2739
Epoch 2/100
143/143 [==============================] - 85s 599ms/step - loss: 1.0336 - accuracy: 0.4852 - val_loss: 1.1105 - val_accuracy: 0.3983
Epoch 3/100
143/143 [==============================] - 84s 589ms/step - loss: 1.0057 - accuracy: 0.5183 - val_loss: 1.1206 - val_accuracy: 0.4129
Epoch 4/100
143/143 [==============================] - 84s 587ms/step - loss: 1.0521 - accuracy: 0.4692 - val_loss: 1.1225 - val_accuracy: 0.3709
Epoch 5/100
143/143 [==============================] - 84s 587ms/step - loss: 1.0249 - accuracy: 0.5041 - val_loss: 1.1237 - val_accuracy: 0.3541
Epoch 6/100
143/143 [==============================] - 84s 586ms/step - loss: 1.0170 - accuracy: 0.5168 - val_loss: 1.0808 - val_accuracy: 0.4028
Epoch 7/100
143/143 [==============================] - 85s 591ms/step - loss: 1.0444 - accuracy: 0.4533 - val_loss: 1.1509 

In [ ]:

# Plot the training and validation loss
plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()